In [3]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/behrooz.filzadeh/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "behrooz.filzadeh"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "5c9bbb8ded87382fe64c827cd354277fbf35e412"


In [4]:
os.chdir("E:\\datascienceproject")
%pwd

'E:\\datascienceproject'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path
    all_params : dict
    metric_file_name : Path
    target_column : str
    mlflow_uri : str


In [9]:
from src.datascienece.constants import *
from src.datascienece.utils.common import read_yaml, create_directories,save_json


In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    def get_mode_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/behrooz.filzadeh/datascienceproject.mlflow"
        )
        return model_evaluation_config

In [13]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib





e:\datascienceproject\datascienceproject\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    def eval_metric(self, actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        return rmse, mae, r2
    def log_into_mlflow(self):
      test_data = pd.read_csv(self.config.test_data_path)
      model = joblib.load(self.config.model_path)
      test_x = test_data.drop([self.config.target_column], axis=1) 
      test_y = test_data[self.config.target_column]
      
      mlflow.set_tracking_uri(self.config.mlflow_uri)
      tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
      
      with mlflow.start_run():
        predicted_qualities = model.predict(test_x)
        rmse, mae, r2 = self.eval_metric(test_y, predicted_qualities)
        
        #saving metrics as local
        scores = {
            "rmse": rmse,
             "mae": mae,
             "r2": r2}
        save_json(path = Path(self.config.metric_file_name), data = scores)
        
        mlflow.log_params(self.config.all_params)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        if tracking_url_type_store != "file":
          mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetWineModel")
        else:
          mlflow.sklearn.log_model(model, "model")

In [19]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_mode_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-02-24 18:46:22,707: INFO: common]: yaml file: config\config.yaml loaded successfully
[2026-02-24 18:46:22,709: INFO: common]: yaml file: parameters.yaml loaded successfully
[2026-02-24 18:46:22,713: INFO: common]: yaml file: schema.yaml loaded successfully
[2026-02-24 18:46:22,715: INFO: common]: Directory created at: artifacts
[2026-02-24 18:46:22,716: INFO: common]: Directory created at: artifacts/model_evaluation
[2026-02-24 18:46:23,931: INFO: common]: json file saved at: artifacts\model_evaluation\metric.json


2026/02/24 18:46:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/24 18:46:26 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html
Successfully registered model 'ElasticNetWineModel'.
2026/02/24 18:46:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetWineModel, version 1
Created version '1' of model 'ElasticNetWineModel'.


🏃 View run wise-grub-995 at: https://dagshub.com/behrooz.filzadeh/datascienceproject.mlflow/#/experiments/0/runs/1ce28ee531a448a2ae74d5aef33c9f6d
🧪 View experiment at: https://dagshub.com/behrooz.filzadeh/datascienceproject.mlflow/#/experiments/0
